# Introduction

This tutorial demonstrates basic steps involved in processing electron
backscatter diffraction (EBSD) patterns before crystal structure determination
(indexing) or other analysis.

For learning purposes, we recommend to use this notebook alongside our [user
guide](https://kikuchipy.org).

The data consists of 29 800 EBSD patterns from recrystallized polycrystalline
Nickel. It is available [via Zenodo](https://zenodo.org/record/3265037).

This functionality has been checked to run in kikuchipy-0.2.0 (May 2020). Bugs
are always possible, do not trust the code blindly, and if you experience any
issues, report them [in our issue
tracker](https://github.com/kikuchipy/kikuchipy-demos/issues).

Access all methods' documentation (docstrings) when the cursor is within the
function parantheses, like `function(CURSOR HERE)`, by holding `SHIFT` and
pressing `TAB`.

# Contents

1. [Loading and data inspection](#1)
2. [Background correction](#2)  
    1. [Static background correction](#2.1)  
    2. [Dynamic background correction](#2.2)
3. [Neighbour pattern averaging](#3)
4. [Further processing](#4)  
    1. [Intensity normalization](#4.1)  
    2. [Intensity rescaling/contrast stretching](#4.2)  
    3. [Filtering in the frequency domain (FFT filtering)](#4.3)  
    4. [Adaptive histogram equalization](#4.4)  

Set the interactive [Matplotlib backend](https://matplotlib.org/3.2.1/tutorials/introductory/usage.html#backends)
(`%matplotlib qt5`), import kikuchipy and other necessary libraries

In [3]:
%matplotlib qt5
import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
import kikuchipy as kp

# 1. Loading and data inspection <a class="anchor" id="1"></a>

Load the EBSD data

In [4]:
data = "/home/hakon/phd/data/ni/2020/1/nordif/Pattern.dat"
s = kp.load(data, lazy=False)  # lazy=True to not load into memory before calling s.compute()

OSError: No filename matches '/home/hakon/phd/data/ni/2020/1/nordif/Pattern.dat'.

Inspect the EBSD object `s`

In [5]:
s

NameError: name 's' is not defined

Inspect the data type

In [6]:
s.data.dtype

NameError: name 's' is not defined

Inspect the metadata associated with the EBSD object `s`

In [7]:
s.metadata

NameError: name 's' is not defined

Set important experimental parameters, like the pattern centre (in EMsoft's convention)

In [8]:
s.set_experimental_parameters(
    xpc=38.86,
    ypc=132.65,
    zpc=16986.48,
    binning=8
)

NameError: name 's' is not defined

See how this changed the metadata of the EBSD metadata node

In [9]:
ebsd_node = kp.signals.util.metadata_nodes("ebsd")

In [10]:
s.metadata.get_item(ebsd_node)

NameError: name 's' is not defined

Set another metadata item and check that

In [11]:
s.metadata.set_item("General.title", "Recrystallized Ni")

NameError: name 's' is not defined

In [12]:
s.metadata

NameError: name 's' is not defined

In [13]:
s

NameError: name 's' is not defined

Inspect the signal (detector) and navigation (sample) axes

In [14]:
s.axes_manager

NameError: name 's' is not defined

Set the detector pixel size (and the detector origin to the detector centre)

In [15]:
s.set_detector_calibration(delta=70)  # Microns

NameError: name 's' is not defined

Inspect the axes manager again (note the signal scale)

In [16]:
s.axes_manager

NameError: name 's' is not defined

Plot the patterns

In [17]:
s.plot()

NameError: name 's' is not defined

If we want to only operate on or inspect parts of a data set, e.g. a square
from an upper left pattern (row, col) = (10, 20) to a bottom right pattern
(row, col) = (50, 70)

In [18]:
#s = s.inav[10:50, 20:70]

# 2. Background correction <a class="anchor" id="2"></a>

## 2.A Static background correction <a class="anchor" id="2.1"></a>

Inspect the static background pattern

In [19]:
static_bg = s.metadata.get_item(ebsd_node + ".static_background")

NameError: name 's' is not defined

In [20]:
plt.imshow(static_bg)

NameError: name 'static_bg' is not defined

Remove the static background from each pattern by subtracting by it, but keeping relative intensities between the patterns

In [21]:
s.remove_static_background(operation="subtract", relative=True)

NameError: name 's' is not defined

Inspect the static background corrected patterns

In [22]:
s.plot()

NameError: name 's' is not defined

## 2.B Dynamic background correction <a class="anchor" id="2.2"></a>

Before we remove the large scale variations on the detector (dynamic background), we want to inspect it. It can be obtained by Gaussian blurring in real or frequency space 

In [23]:
dynamic_bg = s.get_dynamic_background(
    filter_domain="frequency",  # or spatial
    std=8,
    truncate=4,
    dtype_out=np.float32,  # Takes up 4 times the RAM!
)

NameError: name 's' is not defined

Inspect the dynamic background

In [24]:
dynamic_bg.plot()

NameError: name 'dynamic_bg' is not defined

Remove the dynamic background from each pattern by subtracting by it (of course loosing relative intensities between the patterns)

In [25]:
s.remove_dynamic_background(
    operation="subtract",
    filter_domain="frequency",
    std=8,
    truncate=4,
)

NameError: name 's' is not defined

Inspect the dynamic background corrected patterns

In [26]:
s.plot()

NameError: name 's' is not defined

# 3. Neighbour pattern averaging <a class="anchor" id="3"></a>

Create an averaging kernel

In [27]:
w_cross = kp.filters.Window(window="circular", shape=(3, 3))

Inspect the kernel and plot it

In [28]:
w_cross.plot()

(<Figure size 640x480 with 2 Axes>,
 <matplotlib.colorbar.Colorbar at 0x7fe4d5236430>)

In [29]:
w_cross

Window (3, 3) circular
[[0. 1. 0.]
 [1. 1. 1.]
 [0. 1. 0.]]

Create a Gaussian averaging kernel

In [30]:
w_gauss = kp.filters.Window(window="gaussian", std=1)

Inspect it

In [31]:
w_gauss

Window (3, 3) gaussian
[[0.3679 0.6065 0.3679]
 [0.6065 1.     0.6065]
 [0.3679 0.6065 0.3679]]

Average each pattern (in the window centre, `1.`) with the neighbouring patterns with the window coefficients in `w_gauss` (by performing spatial correlation)

In [32]:
s.average_neighbour_patterns(window=w_gauss)

NameError: name 's' is not defined

Inspect the averaged patterns

In [33]:
s.plot()

NameError: name 's' is not defined

# 4. Further processing <a class="anchor" id="4"></a>

## 4.A Intensity normalization <a class="anchor" id="4.1"></a>

Normalize the patterns intensities to a mean of zero and a standard deviation of one

In [34]:
s.change_dtype(np.float32)  # Or passing dtype_out=np.float32 to normalize_intensity()

NameError: name 's' is not defined

In [35]:
s.normalize_intensity()

NameError: name 's' is not defined

Inspect the normalized patterns

In [36]:
s.plot()

NameError: name 's' is not defined

## 4.B Intensity rescaling/contrast stretching <a class="anchor" id="4.2"></a>

Rescale pattern intensities to fill the entire data type range of the `uint16` data type

In [37]:
s.rescale_intensity(dtype_out=np.uint16)

NameError: name 's' is not defined

Inspect the rescaled patterns

In [38]:
s.plot()

NameError: name 's' is not defined

Perform contrast stretching on a copy of the data

In [39]:
s2 = s.deepcopy()  # Double the memory!

NameError: name 's' is not defined

In [40]:
s2.rescale_intensity(percentiles=(0.5, 99.5))

NameError: name 's2' is not defined

Compare the patterns before and after contrast stretching

In [41]:
hs.plot.plot_signals([s, s2])

NameError: name 's' is not defined

## 4.C Filtering in the frequency domain <a class="anchor" id="4.3"></a>

Create a Gaussian lowpass transfer function of pattern shape to remove high frequency components (noise) in the patterns

In [42]:
pattern_shape = s.axes_manager.signal_shape

NameError: name 's' is not defined

In [43]:
pattern_shape

NameError: name 'pattern_shape' is not defined

In [44]:
w_low = kp.filters.Window(
    window="lowpass",
    cutoff=22,
    cutoff_width=10,
    shape=pattern_shape
)

NameError: name 'pattern_shape' is not defined

Inspect the filter

In [45]:
plt.figure()
plt.imshow(w_low)

NameError: name 'w_low' is not defined

Create a Gaussian highpass transfer function of pattern shape to remove low frequency components (large scale variations,
a variant of dynamic background correction) in the pattern

In [46]:
w_high = kp.filters.Window(
    window="highpass",
    cutoff=3,
    cutoff_width=2,
    shape=pattern_shape
)

NameError: name 'pattern_shape' is not defined

Inspect the filter

In [47]:
plt.figure()
plt.imshow(w_high)

NameError: name 'w_high' is not defined

Create a combined Gaussian high and low pass filter

In [48]:
w = w_low * w_high

NameError: name 'w_low' is not defined

Inspect the combined filter

In [49]:
plt.figure()
plt.imshow(w)

NameError: name 'w' is not defined

Apply the filter in the frequency domain on a copy of the data

In [50]:
s3 = s.deepcopy()

s3.fft_filter(
    transfer_function=w,
    function_domain="frequency",  # It is already a transfer function
    shift=True,
)

NameError: name 's' is not defined

Compare the patterns before and after FFT filtering

In [51]:
hs.plot.plot_signals([s, s3])

NameError: name 's' is not defined

## 4.D Adaptive histogram equalization <a class="anchor" id="4.4"></a>

Perform adaptive histogram equalization (like done in EMsoft's dictionary indexing program `EMEBSDDI`) on a copy of the data

In [52]:
s4 = s.deepcopy()

NameError: name 's' is not defined

In [53]:
s4.adaptive_histogram_equalization()

NameError: name 's4' is not defined

Plot the two data sets before and after equalization side-by-side

In [54]:
hs.plot.plot_signals([s, s4])

NameError: name 's' is not defined